Here we perform PCA regression on the data matrix assembled and saved in the  "saved_data" folder

In the later part of the code, PCA is performed on moving windows over the entire range of the data

In [1]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
import allantools
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



In [2]:
#import data matrix
#fpath = '../saved_data/week3_fourth_cycle/demeaned_data_matrix_fourth_cycle_week_3'
fpath = '../saved_data/week1_cycle1/demeaned_data_matrix_first_cycle_week1'

df = pd.read_csv(fpath, sep=',', header=0)

In [3]:
# slice the df into X and y
col_list = df.columns

X = df[col_list[1:]]
y = df[col_list[0]]; y = y+273.15

In [ ]:
df[col_list[0]].plot()

In [5]:
y_lda = (np.floor(y)).astype(int)

In [6]:
# split the data
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_lda_train, y_lda_test = X[:195000], X[195000:], y_lda[:195000], y_lda[195000:]

x_label = X.columns.astype('float').values

In [ ]:
plt.plot(X_train.index.values, y_lda_train.values, 'k')
plt.plot(X_test.index.values, y_lda_test.values, 'r')



In [8]:
# perform LDA on dataset to isolate the temperature mode
# from prev EDA we know the first mode is what we need to isolate

n_comps = 3
# # Create a Dask PCA object
lda = LinearDiscriminantAnalysis(n_components=n_comps, solver='svd')


In [ ]:
# Fit the LDA model
lda.fit(X_train, y_lda_train) 

In [ ]:
# isolate lda mode's top mode

lda_mode  = lda.scalings_[:, 0]
plt.plot(x_label, lda_mode)

# isolate weights for the first mode over the entire range

lda_weights = lda.transform(X)[:, 0]

plt.plot(lda_weights)

In [ ]:
lda_mode.reshape(-1,1).shape

In [ ]:
lda_weights.reshape(1,-1).shape

In [ ]:
# corrected data set

A = np.array(lda_mode).reshape(1, -1)*np.array(lda_weights.reshape(-1, 1))
X1 = X - A

plt.plot(x_label, X1.iloc[0, :].values)                                               

In [ ]:
plt.plot(X1.T);

In [15]:
X1_train, X1_test, y1_train, y1_test = X1[:195000], X1[195000:], y[:195000], y[195000:]


In [ ]:
# specify the number of componenets
n_comps = 10
# # Create a Dask PCA object
pca_train = PCA(n_components=n_comps, svd_solver='auto')

# Fit the PCA model
pca_train.fit_transform(X1_train) 


In [ ]:
# Create scree plot
# Get explained variance ratio
explained_variance_ratio = pca_train.explained_variance_ratio_

export_name = '../saved_data/sensor_2_week_1_first_cycle_screeplot.png'

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.grid()
#plt.savefig(export_name, dpi=700)
plt.show()

In [18]:
#plt.plot(pca_train.components_.T[:, 0])

In [ ]:
for i in range(n_comps):
    plt.plot( pca_train.components_[i])
    plt.title('mode {}'.format(i))
    plt.show()

In [20]:
loadings_train = (X_train@pca_train.components_.T)

In [ ]:
for i in range(n_comps):
    plt.plot(loadings_train[i].values);
    plt.title('mode {}'.format(i))
    plt.show()

In [22]:
#### create a moving average PCA with time window growing with each iteration 
#### at each step save the modes and their loadings to  file


#components_gw = []
#loadings_gw = []
w_size = []
training_error, testing_error = [], []
lr = LinearRegression()

def growing_window_pca_regression(data, y , window_size=10000):
    pca_d = PCA(n_components=10)
    print('test')
    for start in range(0, data.shape[0] - window_size + 1, window_size):
        print(start)
        window = data.iloc[0:start + window_size, :]
        w_size.append(start+window_size)
        y_tr =  y.iloc[0:start + window_size]
        y_tt =  y.iloc[start + window_size:]
        pca_d.fit(window)
        loadings = (data@pca_d.components_.T); 
        lr.fit(window, y_tr)
        y_pred_training = lr.predict(window)
        y_pred_testing = lr.predict(data.iloc[start + window_size:, :])
        training_err = np.round(np.sqrt(mean_squared_error(y_tr, y_pred_training)), 3)
        testing_err = np.round(np.sqrt(mean_squared_error(y_tt, y_pred_testing)), 3)
        training_error.append(training_err)
        testing_error.append(testing_err)
        print('training error is {} and the testing error is {}'.format(training_err, testing_err))
  
        

In [ ]:
y[150000]

In [ ]:
growing_window_pca_regression(X1, y , window_size=30000)

In [ ]:
df_gw = pd.DataFrame(list(zip(w_size, training_error, testing_error)))
df_gw.columns = ['w_size', 'training_error', 'testing_error']

plt.plot(df_gw.w_size.values, df_gw.training_error.values, 'x')
plt.plot(df_gw.w_size.values, df_gw.testing_error.values, 'r^')

In [25]:
#### create a moving average PCA with time window growing with each iteration 
#### at each step save the modes and their loadings to  file


components_gw = []
loadings_gw = []
w_size = []

def growing_window_pca(data, window_size=10000):
    pca_d = PCA(n_components=10)
    print('test')
    for start in range(0, data.shape[0] - window_size + 1, window_size):
        print(start)
        window = data.iloc[0:start + window_size, :]
        w_size.append(start+window_size)
        #plt.plot(y.iloc[0:start + window_size]); plt.show()
        pca_d.fit(window)
        components_gw.append(pca_d.components_)
        loadings = (data@pca_d.components_.T);  
        loadings_gw.append(loadings)
        


In [ ]:
growing_window_pca(X1, window_size=30000)  #10000

In [ ]:
w_size[5]
#components_gw[6]

In [ ]:
plt.plot(components_gw[6][0])

In [29]:
time_step = 5
mode_0 = np.array(components_gw[time_step][0]).reshape(1, -1)*np.array(loadings_gw[time_step][0]).reshape(-1, 1)
mode_1 = np.array(components_gw[time_step][1]).reshape(1, -1)*np.array(loadings_gw[time_step][1]).reshape(-1, 1)
mode_2 = np.array(components_gw[time_step][2]).reshape(1, -1)*np.array(loadings_gw[time_step][2]).reshape(-1, 1)

mode_3 = np.array(components_gw[time_step][3]).reshape(1, -1)*np.array(loadings_gw[time_step][3]).reshape(-1, 1)
mode_4 = np.array(components_gw[time_step][4]).reshape(1, -1)*np.array(loadings_gw[time_step][4]).reshape(-1, 1)
mode_5 = np.array(components_gw[time_step][5]).reshape(1, -1)*np.array(loadings_gw[time_step][5]).reshape(-1, 1)

mode_6 = np.array(components_gw[time_step][6]).reshape(1, -1)*np.array(loadings_gw[time_step][6]).reshape(-1, 1)
mode_7 = np.array(components_gw[time_step][7]).reshape(1, -1)*np.array(loadings_gw[time_step][7]).reshape(-1, 1)
mode_8 = np.array(components_gw[time_step][8]).reshape(1, -1)*np.array(loadings_gw[time_step][8]).reshape(-1, 1)
mode_9 = np.array(components_gw[time_step][9]).reshape(1, -1)*np.array(loadings_gw[time_step][9]).reshape(-1, 1)

In [30]:
B = X1 - mode_0 - mode_1 - mode_2  
C = X1 - mode_0 - mode_1 - mode_2  - mode_3 - mode_4 - mode_5 #- mode_6- mode_7 - mode_8 - mode_6
D = X1 - mode_0 - mode_1 - mode_2  - mode_3 - mode_4 - mode_5 - mode_6- mode_7 - mode_8# - mode_9


In [ ]:
plt.plot(B.iloc[0, :]); plt.ylim(-1000, 3000);plt.show()

In [ ]:
plt.plot(C.iloc[0,:]); plt.ylim(-2000, 2000); plt.show()
plt.plot(C.iloc[100000,:]); plt.ylim(-2000, 2000); plt.show()


In [ ]:
plt.plot(D.iloc[0,:]); plt.ylim(-2000, 2000); 

In [ ]:
# specify the number of componenets
n_comps = 10
# # Create a Dask PCA object
pca_ = PCA(n_components=n_comps, svd_solver='auto')

# Fit the PCA model
#X_reduced_train = pca.fit_transform(X_train) 
pca_.fit_transform(B) 

In [ ]:
for i in range(n_comps):
    plt.plot(pca_.components_[i,:].T);
    plt.title('mode {}'.format(i))
    plt.show()

In [ ]:
loadings_ = B@pca_.components_.T
for i in range(n_comps):
    plt.plot(loadings_[i]);    
    plt.title('loading for mode {} after subtracting top 3 modes'.format(i))
    plt.show()

In [ ]:
for i in range(n_comps):
    pd.plotting.autocorrelation_plot(loadings_[i])
    plt.title('ACF plot for mode {}'.format(i))
    plt.show()

In [ ]:

pd.plotting.autocorrelation_plot(y)
plt.title('ACF plot for temperature')

In [ ]:
# specify the number of componenets
n_comps = 10
# # Create a Dask PCA object
pca_c = PCA(n_components=n_comps, svd_solver='auto')

# Fit the PCA model
#X_reduced_train = pca.fit_transform(X_train) 
pca_c.fit_transform(C) 

In [ ]:
for i in range(n_comps):
    plt.plot(pca_.components_[i,:].T);
    plt.title('mode {}'.format(i))
    plt.show()

In [ ]:
loadings_c = C@pca_c.components_.T
for i in range(n_comps):
    plt.plot(loadings_c[i]);    
    plt.title('loading for mode {}'.format(i))
    plt.show()

In [ ]:
for i in range(n_comps):
    pd.plotting.autocorrelation_plot(loadings_c[i])
    plt.title('ACF plot for mode {} after subtracting top 7 modes'.format(i))
    plt.show()

In [ ]:
# specify the number of componenets
n_comps = 10
# # Create a Dask PCA object
pca_d = PCA(n_components=n_comps, svd_solver='auto')

# Fit the PCA model
#X_reduced_train = pca.fit_transform(X_train) 
pca_d.fit_transform(D) 

# compute loadings
loadings_d = D@pca_d.components_.T
for i in range(n_comps):
    plt.plot(loadings_d[i]);    
    plt.title('loading for mode {}'.format(i))
    plt.show()

In [ ]:
for i in range(n_comps):
    pd.plotting.autocorrelation_plot(loadings_d[i])
    plt.title('ACF plot for mode {}'.format(i))
    plt.show()

In [ ]:
# perform Adev computation mode by mode on loadings from pca on mean centered data


for j in range(n_comps):
    ### amplitude
    yd = loadings_gw[time_step][j]/loadings_gw[time_step][j][0]  
    #plt.plot(yd);    
    #plt.title('normalized loading for mode {}'.format(j)) #after subtracting top 3 modes
    #plt.show()
    t= np.logspace(0,10,100)
    r=1/14 # sample rate
    (t2, ad, ade, adn) = allantools.adev(yd, rate=r, data_type="freq", taus='all')
    fig = plt.loglog(t2, ad)
    plt.title('ADEV for mode {}'.format(j))
    plt.xlabel('log time')
    plt.figure()

In [ ]:
# perform Adev computation mode by mode on loadings from pca perfromed on data with top 3 modes subtracted

for j in range(n_comps):
    ### amplitude
    yd = loadings_[j]/loadings_[j][0]  
    #plt.plot(yd);    
    #plt.title('normalized loading for mode {}'.format(j)) #after subtracting top 3 modes
    #plt.show()
    t= np.logspace(0,10,100)
    r=1/14 # sample rate
    (t2, ad, ade, adn) = allantools.adev(yd, rate=r, data_type="freq", taus='all')
    fig = plt.loglog(t2, ad)
    plt.title('ADEV for mode {}'.format(j))
    plt.xlabel('log time')
    plt.figure()

In [ ]:
# perform Adev computation mode by mode on loadings from pca perfromed on data with top 3 modes subtracted

for j in range(n_comps):
    ### amplitude
    yd = loadings_c[j]/loadings_[j][0]  
    #plt.plot(yd);    
    #plt.title('normalized loading for mode {}'.format(j)) #after subtracting top 3 modes
    #plt.show()
    t= np.logspace(0,10,100)
    r=1/14 # sample rate
    (t2, ad, ade, adn) = allantools.adev(yd, rate=r, data_type="freq", taus='all')
    fig = plt.loglog(t2, ad)
    plt.title('ADEV for mode {}'.format(j))
    plt.xlabel('log time')
    plt.figure()

In [ ]:
# perform Adev computation mode by mode on loadings from pca perfromed on data with top 3 modes subtracted

for j in range(n_comps):
    ### amplitude
    yd = loadings_d[j]/loadings_[j][0]  
    #plt.plot(yd);    
    #plt.title('normalized loading for mode {}'.format(j)) #after subtracting top 3 modes
    #plt.show()
    t= np.logspace(0,10,100)
    r=1/14 # sample rate
    (t2, ad, ade, adn) = allantools.adev(yd, rate=r, data_type="freq", taus='all')
    fig = plt.loglog(t2, ad)
    plt.title('ADEV for mode {}'.format(j))
    plt.xlabel('log time')
    plt.figure()

In [40]:
#### subtract loadings_ from B  

time_step = 5
component_0 = pca_.components_[0]
mode_B_0 = np.array(component_0[0]).reshape(1, -1)*np.array(loadings_[0]).reshape(-1, 1)

In [41]:
BB = B - mode_B_0 

In [ ]:
plt.plot(BB.iloc[0, :])


In [ ]:
# specify the number of componenets
n_comps = 10
# # Create a Dask PCA object
pca_bb = PCA(n_components=n_comps, svd_solver='auto')

# Fit the PCA model
#X_reduced_train = pca.fit_transform(X_train) 
pca_bb.fit_transform(BB) 

for i in range(n_comps):
    plt.plot(pca_bb.components_[i,:].T);
    plt.title('mode {}'.format(i))
    plt.show()



# compute loadings
loadings_bb = BB@pca_bb.components_.T
for i in range(n_comps):
    plt.plot(loadings_bb[i]);    
    plt.title('loading for mode {}'.format(i))
    plt.show()

In [ ]:
# perform Adev computation mode by mode on loadings from pca perfromed on data with top 3 modes subtracted

for j in range(n_comps):
    ### amplitude
    yd = loadings_bb[j]/loadings_bb[j][0]  
    #plt.plot(yd);    
    #plt.title('normalized loading for mode {}'.format(j)) #after subtracting top 3 modes
    #plt.show()
    t= np.logspace(0,10,100)
    r=1/14 # sample rate
    (t2, ad, ade, adn) = allantools.adev(yd, rate=r, data_type="freq", taus='all')
    fig = plt.loglog(t2, ad)
    plt.title('ADEV for mode {}'.format(j))
    plt.xlabel('log time')
    plt.figure()

In [ ]:
'''
# =============================================================================
# GP w/.o scaling
# =============================================================================
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, ConstantKernel, DotProduct

kernel_ = RBF(length_scale = 1, length_scale_bounds = (1e-3, 100))  + WhiteKernel() + ConstantKernel(constant_value = 1, constant_value_bounds = (1e-3, 1e6))
gp_model = GaussianProcessRegressor(kernel=kernel_)

# fit model
gp_model.fit(loadings_train[:, :], y_train)  # Fit the GP on the current batch
'''

In [34]:
#y_pred_train, std_pred_train = gp_model.predict(loadings_train , return_std=True)

In [35]:
#df_gp_train = pd.DataFrame({'y_pred_train':y_pred_train, 'std_pred_train':std_pred_train})
#df_gp_train.to_csv('../saved_data/gp_training_output')

In [36]:
#plt.plot(std_pred_train, y_train.to_numpy())

In [37]:
#X_reduced_test = pca.transform(X_test)

In [38]:
#y_pred_test, std_pred_test = gp_model.predict(X_reduced_test , return_std=True)

In [39]:
#df_gp_train = pd.DataFrame({'y_pred_test':y_pred_test, 'std_pred_test':std_pred_test,})
#df_gp_train.to_csv('../saved_data/gp_testing_output')

In [40]:
#plt.plot(y_pred_test, y_test,'x')

In [ ]:
#plt.plot((y_test - y_pred)/std_pred, 'x')
plt.plot(y_test/30, 'ro')

In [42]:
#X_reduced_all = pca.transform(X)
#y_pred_all, std_pred_all = gp_model.predict(X_reduced_all , return_std=True)
#df_gp_train = pd.DataFrame({'y_pred_all':y_pred_all, 'std_pred_all':std_pred_all,})
#df_gp_train.to_csv('../saved_data/gp_all_output')